In [1]:
import pandas as pd

In [2]:
newest_gw = 3

In [13]:
fpl_players_new_season = pd.read_csv(f"C:/Users/erknud3/fpl-optimization/model/data/New_Season_Data/fpl_players_new_season_gw{newest_gw}.csv")

teams_pred_npxG = pd.read_csv(f"C:/Users/erknud3/fpl-optimization/model/data/Prediction_Data/teams_pred_npxG_gw{newest_gw}.csv")
teams_pred_npxGC = pd.read_csv(f"C:/Users/erknud3/fpl-optimization/model/data/Prediction_Data/teams_pred_npxGC_gw{newest_gw}.csv")

gc_probs = pd.read_csv(f"C:/Users/erknud3/fpl-optimization/model/data/Prediction_Data/GC_probabilities.csv")

In [11]:
fpl_players_new_season.keys()

Index(['fbref', 'fpl_id', 'first_name', 'second_name', 'Player', 'web_name',
       'Age', 'team_id', 'team_name', 'short_name', 'element_type', 'position',
       'Seasons_count', 'now_cost', 'tsb', 'MP', 'Starts', 'Min', '90s',
       'npxG', 'xAG', 'MP_new', '90s_new', 'npxG_new', 'xAG_new',
       'weighted_npxG', 'weighted_xAG'],
      dtype='object')

In [21]:
print(gc_probs.head(20))

    team_id            Squad  1_0_goals  1_1_goals  1_2_goals  1_4_goals  \
0         1          Arsenal     0.5712     0.3199     0.0896     0.0023   
1         2      Aston Villa     0.2299     0.3380     0.2484     0.0447   
2         3      Bournemouth     0.2417     0.3432     0.2437     0.0409   
3         4        Brentford     0.3606     0.3678     0.1876     0.0163   
4         5         Brighton     0.2671     0.3526     0.2327     0.0338   
5         6          Chelsea     0.1845     0.3118     0.2635     0.0627   
6         7   Crystal Palace     0.2299     0.3380     0.2484     0.0447   
7         8          Everton     0.2725     0.3543     0.2303     0.0324   
8         9           Fulham     0.1686     0.3002     0.2672     0.0705   
9        10     Ipswich Town     0.0714     0.1884     0.2487     0.1444   
10       11   Leicester City     0.1588     0.2922     0.2688     0.0759   
11       12        Liverpool     0.4819     0.3518     0.1284     0.0057   
12       13 

In [18]:
# Define position multipliers
position_multipliers = {1: 10, 2: 6, 3: 5, 4: 4}

# Select the relevant columns
columns_to_keep = ['fpl_id', 'Player', 'web_name', 'Age', 'team_id', 'team_name', 'element_type', 'position', 'now_cost', 'tsb', 'Min', '90s', 'MP_new', '90s_new', 'weighted_npxG']
player_xp_goals = fpl_players_new_season[columns_to_keep].copy()

# Iterate over each gameweek and calculate expected points from goals
for gw in range(1, 39):
    gw_column = str(gw)  # Convert the gameweek number to string for column access
    player_xp_goals[gw_column] = player_xp_goals.apply(
        lambda row: row['weighted_npxG'] * teams_pred_npxG.loc[teams_pred_npxG['team_id'] == row['team_id'], gw_column].values[0] * position_multipliers[row['element_type']],
        axis=1
    )

# Optional: Round the values to 2 decimal places
player_xp_goals = player_xp_goals.round(2)

# Save the resulting DataFrame to a CSV file
file_path = f"C:/Users/erknud3/fpl-optimization/model/data/Prediction_Data/player_xp_goals_gw{newest_gw}.csv"
player_xp_goals.to_csv(file_path, index=False)

print(f"Player expected points from goals successfully saved to {file_path}")


Player expected points from goals successfully saved to C:/Users/erknud3/fpl-optimization/model/data/Prediction_Data/player_xp_goals_gw3.csv


In [19]:
# Select the relevant columns
columns_to_keep = ['fpl_id', 'Player', 'web_name', 'Age', 'team_id', 'team_name', 'element_type', 'position', 'now_cost', 'tsb', 'Min', '90s', 'MP_new', '90s_new', 'weighted_xAG']
player_xp_assists = fpl_players_new_season[columns_to_keep].copy()

# Iterate over each gameweek and calculate expected points from goals
for gw in range(1, 39):
    gw_column = str(gw)  # Convert the gameweek number to string for column access
    player_xp_assists[gw_column] = player_xp_assists.apply(
        lambda row: row['weighted_xAG'] * teams_pred_npxG.loc[teams_pred_npxG['team_id'] == row['team_id'], gw_column].values[0] * 3,
        axis=1
    )

# Optional: Round the values to 2 decimal places
player_xp_assists = player_xp_assists.round(2)

# Save the resulting DataFrame to a CSV file
file_path = f"C:/Users/erknud3/fpl-optimization/model/data/Prediction_Data/player_xp_assists_gw{newest_gw}.csv"
player_xp_assists.to_csv(file_path, index=False)

print(f"Player expected points from goals successfully saved to {file_path}")


Player expected points from goals successfully saved to C:/Users/erknud3/fpl-optimization/model/data/Prediction_Data/player_xp_assists_gw3.csv


In [22]:
# Create a new DataFrame for player expected points from clean sheets
columns_to_keep = ['fpl_id', 'Player', 'web_name', 'Age', 'team_name', 'team_id', 'element_type', 'now_cost', 'tsb', 'Min', '90s', 'MP_new', '90s_new']
player_xp_cs = fpl_players_new_season[columns_to_keep].copy()

# Define the points associated with each goal scenario
points_per_goal_scenario = {
    0: {1: 4, 2: 4, 3: 1, 4: 0},  # 4 points for GKP and DEF, 1 point for MID, 0 points for FWD
    1: 0,                         # 0 points for conceding 1 goal
    2: {1: -1, 2: -1, 3: 0, 4: 0}, # -1 points for GKP and DEF, 0 points for MID and FWD
    4: {1: -2, 2: -2, 3: 0, 4: 0}, # -2 points for GKP and DEF, 0 points for MID and FWD
    6: {1: -3, 2: -3, 3: 0, 4: 0}, # -3 points for GKP and DEF, 0 points for MID and FWD
    8: {1: -4, 2: -4, 3: 0, 4: 0}  # -4 points for GKP and DEF, 0 points for MID and FWD
}

# Iterate over each gameweek and calculate expected points from clean sheets
for gw in range(1, 39):
    cs_column = f'{gw}_0_goals'
    gc_1_column = f'{gw}_1_goals'
    gc_2_column = f'{gw}_2_goals'
    gc_4_column = f'{gw}_4_goals'
    gc_6_column = f'{gw}_6_goals'
    gc_8_column = f'{gw}_8_goals'

    # Calculate expected points based on probabilities and the points per scenario
    player_xp_cs[str(gw)] = player_xp_cs.apply(
        lambda row: (
            gc_probs.loc[gc_probs['team_id'] == row['team_id'], cs_column].values[0] * points_per_goal_scenario[0][row['element_type']] +
            gc_probs.loc[gc_probs['team_id'] == row['team_id'], gc_1_column].values[0] * points_per_goal_scenario[1] +
            gc_probs.loc[gc_probs['team_id'] == row['team_id'], gc_2_column].values[0] * points_per_goal_scenario[2][row['element_type']] +
            gc_probs.loc[gc_probs['team_id'] == row['team_id'], gc_4_column].values[0] * points_per_goal_scenario[4][row['element_type']] +
            gc_probs.loc[gc_probs['team_id'] == row['team_id'], gc_6_column].values[0] * points_per_goal_scenario[6][row['element_type']] +
            gc_probs.loc[gc_probs['team_id'] == row['team_id'], gc_8_column].values[0] * points_per_goal_scenario[8][row['element_type']]
        ),
        axis=1
    )

# Optional: Round the values to 2 decimal places
player_xp_cs = player_xp_cs.round(2)

# Save the resulting DataFrame to a CSV file
file_path = f"C:/Users/erknud3/fpl-optimization/model/data/Prediction_Data/player_xp_cs_gw{newest_gw}.csv"
player_xp_cs.to_csv(file_path, index=False)

print(f"Player expected points from clean sheets successfully saved to {file_path}")


Player expected points from clean sheets successfully saved to C:/Users/erknud3/fpl-optimization/model/data/Prediction_Data/player_xp_cs_gw3.csv
